# Generate SR-Lite products

## Workflow Steps
1. View input projections (EVHR-TOA, CCDC, and Cloudmasks)
2. Warp input arrays (as needed) to common EVHR-TOA 30m grid
3. Get the common masked data (the union of each mask)
4. Perform a linear regression to 'fit' the corresponding CCDC bands to the EVHR bands
5. Capture the regression coefficients (e.g., intercept, slope)
6. Apply the coefficients to the original 2m EVHR date to 'predict' Surface Reflectance
7. Aggregate the Surface Reflectance bands into consolidate image (i.e., tif)
8. Create a Cloud Optimized Geotiff (COG) image from stack of processed bands

### Pre-Requisites:
1) TOA files (2m) - generated by EHVR (e.g., /adapt/nobackup/people/iluser/projects/srlite/input/TOA_v2/Yukon_Delta/5-toas)
2) CCDC files (30m) - generated by GEE (e.g,. /adapt/nobackup/people/iluser/projects/srlite/input/CCDC_v2 )
3) Cloudmask files (2m) - generated by Jordan (e.g., /adapt/nobackup/people/iluser/projects/srlite/input/CloudMask
4) Each of above must share a common ID and prefix in the file name (e.g., WV02_20150616_M1BS_103001004351F000)

## Command Line Input Parameters:
#### Mandatory:
**`-toa_dir`**  `<directory path containing EVHR TOA files>`
<br> **`-ccdc_dir`** `<directory path containing CCDC files>`
<br> **`-cloudmask_dir`** `<directory path containing Cloudmask files>`
<br> **`-bandpairs`** `<list of pairs that specify which named bands to compare in the CCDC and EVHR input files>` 
<br> **`-output_dir`** `<destination directory path for SR-Lite products>`

#### Optional (default in *italic*):
**`--regressor`** `<class of regression> ` Options:  *robust* or simple
<br> **`--debug`** `<verbosity level> ` Options:  *0*=No debug, 1=Trace text, 2=Detailed text, 3=Plots & histograms
<br> **`--clean`** `< Remove pre-existing artifacts (e.g, translated files)? > ` Options: True if parameter exists, otherwise *False*


`========================================================================`
<br>`============ SR-Lite Application Code ==================================`
<br>`========================================================================`

## Specify package dependencies path to SR-Lite Python application code 

In [1]:
import os, sys, time, pathlib

sys.path.append('/adapt/nobackup/people/iluser/projects/srlite/src/srlite-0.9.4-04152022-cog/srlite')
from srlite.model.Context import Context
from srlite.model.RasterLib import RasterLib

/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


## SR-Lite Main

In [2]:
# --------------------------------------------------------------------------------
# Main - Generate SR-Lite products
#
# Implementation Note: A Context() class is used to consolidate all run-time information.  
# This Context(), which is continuously updated during the run to reflect the state of the
# application and its inputs and outputs, is passed as the lone parameter between functions.  
# This approach supports parameter expansion at the service (i.e., function) level without
# requiring API changes.  Also, the Context() is serializable by design, allowing it to be
# easily served to worker threads.  This will come in handy in the next phase, when the
# application is distributed across nodes.
# --------------------------------------------------------------------------------
def main():

        # Initialize context
        start_time = time.time()  # record start time
        contextClazz = Context()
        context = contextClazz.getDict()

        # Get handles to plot and raster classes
        plotLib = contextClazz.getPlotLib()
        rasterLib = RasterLib(int(context[Context.DEBUG_LEVEL]), plotLib)

        #  Loop through the EVHR TOA files, generating SR-Lite for each one
        for context[Context.FN_TOA] in sorted(pathlib.Path(context[Context.DIR_TOA]).glob("*.tif")):
            try:
                # Generate file names based on incoming EVHR file and declared suffixes - get snapshot
                start_time = time.time()  # record start time
                context = contextClazz.getFileNames(str(context[Context.FN_TOA]).rsplit("/", 1), context)
                cogname = str(context[Context.DIR_OUTPUT]) + str('/') + \
                          str(context[Context.FN_PREFIX]) + str(Context.FN_SRLITE_SUFFIX)

                # Remove existing SR-Lite output if clean_flag is activated
                fileExists = (os.path.exists(cogname))
                if fileExists and (eval(context[Context.CLEAN_FLAG])):
                    rasterLib.removeFile(cogname, context[Context.CLEAN_FLAG])

                # Proceed if SR-Lite output does not exist
                if not fileExists:

                    #  Warp cloudmask to attributes of EVHR - suffix root name with '-toa_pred_warp.tif')
                    rasterLib.getAttributeSnapshot(context)
                    context[Context.FN_SRC] = str(context[Context.FN_CLOUDMASK])
                    context[Context.FN_DEST] = str(context[Context.FN_WARP])
                    context[Context.TARGET_ATTR] = str(context[Context.FN_TOA])
                    rasterLib.translate(context)
                    rasterLib.getAttributes(str(context[Context.FN_WARP]), "Cloudmask Warp Combo Plot")

                    # Validate that input band name pairs exist in EVHR & CCDC files
                    context[Context.FN_LIST] = [str(context[Context.FN_CCDC]), str(context[Context.FN_TOA])]
                    context[Context.LIST_BAND_PAIR_INDICES] = rasterLib.getBandIndices(context)

                    # Get the common pixel intersection values of the EVHR & CCDC files, then apply CloudMask
                    context[Context.DS_LIST], context[Context.MA_LIST] = rasterLib.getIntersection(context)

                    # Perform regression to capture coefficients from intersected pixels and apply to 2m EVHR
                    context[Context.PRED_LIST] = rasterLib.performRegression(context)

                    # Create COG image from stack of processed bands
                    context[Context.FN_COG] = rasterLib.createImage(context)

                print("\nElapsed Time for " + str(context[Context.FN_COG])  + ': ',
                       (time.time() - start_time) / 60.0)  # time in min            
            
            except FileNotFoundError as exc:
                print('File Not Found - Error details: ', exc)
            except BaseException as err:
                print('Run abended - Error details: ', err)
                sys.exit(1)

        # print("\nElapsed Time for " + str(context[Context.DIR_OUTPUT])  + ': ',
        #        (time.time() - start_time) / 60.0)  # time in min

#### Initialize Inputs - Populate Context() with input parameters and run application

In [3]:
if __name__ == "__main__":
    
    # Supply the command line values for the parameters that are detailed above.  Replace the paths with your local paths, and
    # specify the desired debug level.  Note that when specifying "--debug", "3", combination plots containing images and
    # histograms will be generated for each band that is being processed.  This visualization is very handy for 
    # analyzing artifacts at runtime.  Note, however, that the application takes much longer to finish.
    #
    # By applying the mock patch below to the argv parameter, we allow the main
    # routine to be called from either a Python application or Jupyter Notebook without code changes.
    from unittest.mock import patch
    with patch("sys.argv", 
        ["srlite_cloudmask.ipynb", \
        "-toa_dir", "/adapt/nobackup/people/iluser/projects/srlite/input/TOA_v2/Yukon_Delta/5-toas", 
        "-ccdc_dir", "/adapt/nobackup/people/iluser/projects/srlite/input/CCDC_v2",
        "-cloudmask_dir", "/adapt/nobackup/people/iluser/projects/srlite/input/CloudMask/clouds-binary-pytorch-Yukon_Delta-2022-03-24",
        "-bandpairs", "[['blue_ccdc', 'BAND-B'], ['green_ccdc', 'BAND-G'], ['red_ccdc', 'BAND-R'], ['nir_ccdc', 'BAND-N']]",
        "-output_dir", "/adapt/nobackup/people/iluser/projects/srlite/output/srlite-0.9.4-04102022-translate/04182022/Yukon_Delta",
        "--debug", "0",
        "--regressor", "robust",
        "--clean"
        ]):
   
        run_start_time = time.time()  # record start time
        print(f'Command line executed:    {sys.argv}')

        # Perform SR-Lite in serial fashion (parallel version in progress)
        main()
        
        print("\nTotal Elapsed Time for Run: ", (time.time() - run_start_time) / 60.0)  # time in min

Command line executed:    ['srlite_cloudmask.ipynb', '-toa_dir', '/adapt/nobackup/people/iluser/projects/srlite/input/TOA_v2/Yukon_Delta/5-toas', '-ccdc_dir', '/adapt/nobackup/people/iluser/projects/srlite/input/CCDC_v2', '-cloudmask_dir', '/adapt/nobackup/people/iluser/projects/srlite/input/CloudMask/clouds-binary-pytorch-Yukon_Delta-2022-03-24', '-bandpairs', "[['blue_ccdc', 'BAND-B'], ['green_ccdc', 'BAND-G'], ['red_ccdc', 'BAND-R'], ['nir_ccdc', 'BAND-N']]", '-output_dir', '/adapt/nobackup/people/iluser/projects/srlite/output/srlite-0.9.4-04102022-translate/04182022/Yukon_Delta', '--debug', '0', '--regressor', 'robust', '--clean']

Warping all inputs to the following:
Resolution: 30.0
Extent: [556413.0, 6873199.0, 576637.0, 6987865.0]
Projection: '+proj=utm +zone=3 +datum=WGS84 +units=m +no_defs'
Resampling alg: average

1 of 2: /adapt/nobackup/people/iluser/projects/srlite/input/CCDC_v2/WV02_20110818_M1BS_103001000CCC9000-ccdc.tif
nl: 3822 ns: 674 res: 30.000
0...10...20...30...40